# IWRC Interactive Project Type Breakdown Analysis

## Purpose
This notebook generates **interactive HTML visualizations** that:
1. Analyze 5-year (2020-2024) and 10-year (2015-2024) periods.
2. Break down data by **Total Projects** vs. **Seed Funding (104B)**.
3. Apply **IWRC Branding** (Teal/Olive colors, Montserrat font, Logo).

**Output Directory:** `FINAL_DELIVERABLES_2_backup_20251125_194954 copy 2/visualizations/interactive_breakdown/`

---

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import json
import base64
import os
import re
import warnings
warnings.filterwarnings('ignore')

# IWRC Branding
COLORS = {
    'primary': '#258372',    # Teal
    'secondary': '#639757',  # Olive
    'text': '#54595F',       # Dark Gray
    'accent': '#FCC080',     # Peach/Orange
    'bg': '#F6F6F6'          # Light Gray
}

FONT_FAMILY = "Montserrat, Arial, sans-serif"

# Create output directory
output_dir = '/Users/shivpat/seed-fund-tracking/FINAL_DELIVERABLES_2_backup_20251125_194954 copy 2/visualizations/interactive_breakdown'
os.makedirs(output_dir, exist_ok=True)

# Load Logo (if available)
logo_path = '../../IWRC Logo - Full Color.svg'
logo_base64 = ""
try:
    with open(logo_path, "rb") as image_file:
        logo_base64 = "data:image/svg+xml;base64," + base64.b64encode(image_file.read()).decode('utf-8')
    print("✓ Logo loaded successfully")
except Exception as e:
    print(f"⚠ Could not load logo: {e}")

print(f"✓ Output directory created: {output_dir}")

✓ Logo loaded successfully
✓ Output directory created: /Users/shivpat/seed-fund-tracking/FINAL_DELIVERABLES_2_backup_20251125_194954 copy 2/visualizations/interactive_breakdown


In [2]:
# Load Data
file_path = '../../data/consolidated/IWRC Seed Fund Tracking.xlsx'
df = pd.read_excel(file_path, sheet_name='Project Overview')

# Column Mapping
col_map = {
    'Project ID ': 'project_id',
    'Award Type': 'award_type',
    'Project Title': 'project_title',
    'Project PI': 'pi_name',
    'PI Email': 'pi_email',
    'Academic Institution of PI': 'institution',
    'Award Amount Allocated ($) this must be filled in for all lines': 'award_amount',
    'Number of PhD Students Supported by WRRA $': 'phd_students',
    'Number of MS Students Supported by WRRA $': 'ms_students',
    'Number of Undergraduate Students Supported by WRRA $': 'undergrad_students',
    'Number of Post Docs Supported by WRRA $': 'postdoc_students',
    "Award, Achievement, or Grant\n (This may include awards and achievements for projects from the previous year to this 5-year cycle, so long as they were not already included in last year's report)": 'awards_grants',
    'Monetary Benefit of Award or Achievement (if applicable; use NA if not applicable)': 'monetary_benefit',
    "Description of Award, Achievement, or Grant\n (This may include awards and achievements for projects from the previous year to this 5-year cycle, so long as they were not already included in last year's report)": 'award_description',
    'WRRI Science Priority that Best Aligns with this Project': 'science_priority',
    'Keyword (Primary)': 'keyword_primary',
    'Project Summaries': 'project_summary',
    'Digital Object Identifier (DOI) or other identifier or web page ': 'doi'
}

df_work = df.rename(columns=col_map)

# Clean Student Columns
student_cols = ['phd_students', 'ms_students', 'undergrad_students', 'postdoc_students']
for col in student_cols:
    df_work[col] = pd.to_numeric(df_work[col], errors='coerce').fillna(0)

# Year Extraction Logic
def extract_year(project_id):
    if pd.isna(project_id): return None
    s = str(project_id).strip()
    match = re.search(r'(20\d{2}|19\d{2})', s)
    if match: return int(match.group(1))
    match_fy = re.search(r'FY(\d{2})', s, re.IGNORECASE)
    if match_fy:
        y = int(match_fy.group(1))
        return 2000 + y if y < 100 else y
    return None

df_work['project_year'] = df_work['project_id'].apply(extract_year)

# Follow-on Funding Calculation
def clean_money(val):
    if pd.isna(val): return 0.0
    s = str(val).upper()
    if s in ['NA', 'N/A', 'NONE', '']: return 0.0
    matches = re.findall(r'\$?\s*([\d,]+(?:\.\d{2})?)', s)
    total = 0.0
    for m in matches:
        try: total += float(m.replace(',', ''))
        except: continue
    return total

df_work['money_clean'] = df_work.apply(lambda row: 
    clean_money(row['monetary_benefit']) if clean_money(row['monetary_benefit']) > 0 
    else (clean_money(row['award_description']) if clean_money(row['award_description']) > 0 
    else clean_money(row['awards_grants'])), axis=1)

# --- DATA CLEANING FOR MAP ---
# Normalize Institution Names
def normalize_institution(name):
    if pd.isna(name): return None
    n = str(name).strip()
    # UIUC Variants
    if re.search(r'University of Illinois.*Urbana', n, re.IGNORECASE) or n in ['University of Illinois', 'Univeristy of Illinois', 'University of Illinois ']:
        return 'University of Illinois Urbana-Champaign'
    # SIU Variants
    if re.search(r'Southern Illinois University.*Carbondale', n, re.IGNORECASE) or n == 'Southern Illinois University':
        return 'Southern Illinois University Carbondale'
    # UIC
    if 'Chicago' in n and 'Illinois' in n:
        return 'University of Illinois Chicago'
    return n

df_work['institution_clean'] = df_work['institution'].apply(normalize_institution)

# Filter out rows with NO institution for the map (fixes the 'null' bubble)
df_map_source = df_work.dropna(subset=['institution_clean']).copy()

# --- DEDUPLICATION ---
# Remove duplicate researchers/projects within the same institution
# We keep the first occurrence
df_map_source.drop_duplicates(subset=['institution_clean', 'pi_name', 'project_title'], inplace=True)
print(f"✓ Removed duplicates. Projects count: {len(df_map_source)}")

# Define Tracks (using cleaned data)
df_seed_only = df_work[df_work['award_type'] == 'Base Grant (104b)'].copy()
df_map_seed = df_map_source[df_map_source['award_type'] == 'Base Grant (104b)'].copy()

print(f'✓ Data Loaded & Processed')
print(f'  Total Projects: {len(df_work)}')
print(f'  Projects with Valid Institution: {len(df_map_source)}')
print(f'  Seed Funding (104B): {len(df_seed_only)}')

✓ Removed duplicates. Projects count: 125
✓ Data Loaded & Processed
  Total Projects: 354
  Projects with Valid Institution: 125
  Seed Funding (104B): 142


In [3]:
# --- 1. INTERACTIVE ROI DASHBOARD ---

def create_roi_dashboard():
    # Calculate Metrics for all 4 permutations
    def get_metrics(df_sub, start, end):
        d = df_sub[df_sub['project_year'].between(start, end)]
        inv = d['award_amount'].sum()
        follow = d['money_clean'].sum()
        roi = follow / inv if inv > 0 else 0
        students = d[student_cols].sum().sum()
        return inv, follow, roi, students

    # 10-Year
    t_10_inv, t_10_fol, t_10_roi, t_10_stu = get_metrics(df_work, 2015, 2024)
    s_10_inv, s_10_fol, s_10_roi, s_10_stu = get_metrics(df_seed_only, 2015, 2024)
    
    # 5-Year
    t_5_inv, t_5_fol, t_5_roi, t_5_stu = get_metrics(df_work, 2020, 2024)
    s_5_inv, s_5_fol, s_5_roi, s_5_stu = get_metrics(df_seed_only, 2020, 2024)

    # Create Figure with Subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=("Investment ($)", "Follow-on Funding ($)", "ROI Multiplier", "Students Trained"),
        vertical_spacing=0.15
    )

    # Helper to add traces
    def add_traces(row, col, t_val_10, s_val_10, t_val_5, s_val_5, fmt_str):
        # 10-Year Traces (Visible by default)
        fig.add_trace(go.Bar(
            x=['Total Projects', 'Seed Funding'],
            y=[t_val_10, s_val_10],
            name='10-Year',
            marker_color=[COLORS['primary'], COLORS['secondary']],
            text=[fmt_str.format(t_val_10), fmt_str.format(s_val_10)],
            textposition='auto',
            showlegend=False
        ), row=row, col=col)
        
        # 5-Year Traces (Hidden by default)
        fig.add_trace(go.Bar(
            x=['Total Projects', 'Seed Funding'],
            y=[t_val_5, s_val_5],
            name='5-Year',
            marker_color=[COLORS['primary'], COLORS['secondary']],
            text=[fmt_str.format(t_val_5), fmt_str.format(s_val_5)],
            textposition='auto',
            showlegend=False,
            visible=False
        ), row=row, col=col)

    add_traces(1, 1, t_10_inv, s_10_inv, t_5_inv, s_5_inv, '${:,.0f}')
    add_traces(1, 2, t_10_fol, s_10_fol, t_5_fol, s_5_fol, '${:,.0f}')
    add_traces(2, 1, t_10_roi, s_10_roi, t_5_roi, s_5_roi, '{:.2f}x')
    add_traces(2, 2, t_10_stu, s_10_stu, t_5_stu, s_5_stu, '{:,.0f}')

    # Update Layout with UX Fixes
    fig.update_layout(
        title=dict(
            text="<b>IWRC Impact Dashboard: Total vs. Seed Funding</b>",
            font=dict(size=24, family=FONT_FAMILY, color=COLORS['primary']),
            y=0.95,  # Lower title slightly
            x=0.5,
            xanchor='center',
            yanchor='top'
        ),
        updatemenus=[dict(
            type="buttons",
            direction="left",  # Horizontal buttons
            x=0.0, y=1.15,     # Top Left
            xanchor='left', yanchor='top',
            showactive=True,
            buttons=list([
                dict(label="10-Year (2015-2024)",
                     method="update",
                     args=[{"visible": [True, False] * 4}]),
                dict(label="5-Year (2020-2024)",
                     method="update",
                     args=[{"visible": [False, True] * 4}])
            ]),
            bgcolor=COLORS['bg'],
            font=dict(color=COLORS['text'])
        )],
        template="plotly_white",
        font=dict(family=FONT_FAMILY, color=COLORS['text']),
        height=800,
        margin=dict(t=160, l=50, r=50, b=50) # Increase top margin significantly
    )

    # Add Logo
    if logo_base64:
        fig.add_layout_image(
            dict(
                source=logo_base64,
                xref="paper", yref="paper",
                x=1, y=1.18, # Top Right, slightly higher
                sizex=0.18, sizey=0.18,
                xanchor="right", yanchor="top"
            )
        )

    fig.write_html(f'{output_dir}/interactive_roi_dashboard.html')
    print("✓ Saved interactive_roi_dashboard.html")

create_roi_dashboard()

✓ Saved interactive_roi_dashboard.html


In [4]:
# --- 2. INTERACTIVE MAP (MULTI-SELECT) ---

def create_custom_interactive_map():
    # 1. Prepare Data Structure for JS
    inst_data = {}
    
    # Filter for 2015-2024
    df_sub = df_map_source[df_map_source['project_year'].between(2015, 2024)].copy()
    
    for inst in df_sub['institution_clean'].unique():
        inst_df = df_sub[df_sub['institution_clean'] == inst].sort_values('project_year', ascending=False)
        projects = []
        for _, row in inst_df.iterrows():
            # Link Logic: DOI > Google Scholar
            link = "#"
            if pd.notna(row.get('doi')) and str(row.get('doi')).strip().startswith('http'):
                link = str(row.get('doi')).strip()
            else:
                # Generate Google Scholar Link
                q = str(row['project_title']).replace(' ', '+')
                link = f"https://scholar.google.com/scholar?q={q}"
            
            projects.append({
                "pi": row['pi_name'],
                "email": str(row['pi_email']) if pd.notna(row['pi_email']) else "",
                "title": row['project_title'],
                "summary": str(row['project_summary']) if pd.notna(row['project_summary']) else (str(row['award_description']) if pd.notna(row['award_description']) else "No summary available."),
                "link": link,
                "year": int(row['project_year']) if pd.notna(row['project_year']) else ""
            })
        inst_data[inst] = projects

    # 2. Create Base Plotly Map (Div Only)
    # Coordinates
    coords = {
        'University of Illinois Urbana-Champaign': (40.1020, -88.2272),
        'Southern Illinois University Carbondale': (37.7144, -89.2176),
        'Illinois State University': (40.5101, -88.9930),
        'Northwestern University': (42.0565, -87.6753),
        'University of Illinois Chicago': (41.8708, -87.6505),
        'Governors State University': (41.4485, -87.7156),
        'Western Illinois University': (40.4664, -90.6804),
        'Eastern Illinois University': (39.4833, -88.1754),
        'Northern Illinois University': (41.9351, -88.7735),
        'Loyola University Chicago': (42.0008, -87.6574),
        'DePaul University': (41.9249, -87.6550),
        'Bradley University': (40.6977, -89.6160),
        'Chicago State University': (41.7196, -87.6087),
        'Northeastern Illinois University': (41.9803, -87.7176),
        'Southern Illinois University Edwardsville': (38.7940, -89.9986),
        'Illinois Institute of Technology': (41.8349, -87.6270),
        'Lewis University': (41.6047, -88.0806),
        'Cary Institute of Ecosystem Studies': (41.7850, -73.7300),
        'Illinois State Water Survey': (40.0847, -88.2434),
        'University of Texas at Austin': (30.2849, -97.7341),
        "Basil's Harvest": (40.8, -89.6),
        'Lewis and Clark Community College': (38.9489, -90.1946),
        'Illinois Sustainable Technology Center': (40.0965, -88.2394),
        'National Great Rivers Research & Education Center': (38.8873, -90.1378)
    }
    def get_lat(inst): return coords.get(inst, (40.0, -89.0))[0]
    def get_lon(inst): return coords.get(inst, (40.0, -89.0))[1]

    # Prepare Map Data
    map_rows = []
    for inst, projects in inst_data.items():
        map_rows.append({
            'institution': inst,
            'count': len(projects),
            'lat': get_lat(inst),
            'lon': get_lon(inst)
        })
    df_map = pd.DataFrame(map_rows)

    fig = go.Figure(go.Scattermapbox(
        lat=df_map['lat'], lon=df_map['lon'],
        mode='markers+text',
        marker=go.scattermapbox.Marker(
            size=df_map['count']*3 + 15,
            color=COLORS['primary'],
            opacity=0.9
        ),
        text=df_map['institution'],
        customdata=df_map['institution'],
        textposition="top center",
        hoverinfo='text', 
        hovertext=df_map['institution']
    ))

    fig.update_layout(
        mapbox=dict(
            style="carto-positron",
            zoom=6,
            center=dict(lat=40.0, lon=-89.0),
        ),
        margin=dict(t=0, l=0, r=0, b=0),
        height=800,
        clickmode='event+select' # Enable multi-select
    )

    # Get Plotly Div
    plot_div = pio.to_html(fig, full_html=False, include_plotlyjs='cdn', div_id='map_div')

    # 3. Construct Full HTML with Custom UI
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>IWRC Interactive Map</title>
        <link href="https://fonts.googleapis.com/css2?family=Montserrat:wght@300;400;600&display=swap" rel="stylesheet">
        <style>
            body {{ margin: 0; padding: 0; font-family: 'Montserrat', sans-serif; overflow: hidden; }}
            #container {{ display: flex; height: 100vh; width: 100vw; }}
            #map_container {{ flex-grow: 1; height: 100%; position: relative; }}
            
            /* Side Panel */
            #side_panel {{ 
                width: 0px; 
                background: white; 
                box-shadow: -2px 0 10px rgba(0,0,0,0.1); 
                transition: width 0.3s ease; 
                overflow-y: auto; 
                z-index: 1000;
                display: flex;
                flex-direction: column;
            }}
            #side_panel.open {{ width: 400px; }}
            
            /* Header */
            .panel-header {{ 
                background: {COLORS['primary']}; 
                color: white; 
                padding: 20px; 
                position: sticky; 
                top: 0; 
                z-index: 10;
            }}
            .panel-header h2 {{ margin: 0; font-size: 18px; }}
            .close-btn {{ float: right; cursor: pointer; font-size: 20px; }}
            .clear-btn {{ 
                font-size: 12px; 
                color: white; 
                text-decoration: underline; 
                cursor: pointer; 
                margin-top: 5px; 
                display: inline-block;
            }}

            /* Content */
            .panel-content {{ padding: 20px; }}
            
            /* Institution Group */
            .inst-group {{ margin-bottom: 30px; }}
            .inst-header {{ 
                font-size: 16px; 
                font-weight: 600; 
                color: {COLORS['secondary']}; 
                border-bottom: 2px solid #eee; 
                padding-bottom: 5px; 
                margin-bottom: 10px;
            }}

            /* List Item */
            .researcher-item {{ 
                padding: 10px; 
                border-bottom: 1px solid #f5f5f5; 
                cursor: pointer; 
                transition: background 0.2s;
            }}
            .researcher-item:hover {{ background: #f9f9f9; }}
            .researcher-name {{ font-weight: 600; color: {COLORS['primary']}; font-size: 14px; }}
            .project-title {{ font-size: 12px; color: #666; margin-top: 3px; }}

            /* Detail View */
            .detail-view {{ display: none; }}
            .back-btn {{ 
                cursor: pointer; 
                color: white; 
                margin-bottom: 10px; 
                display: inline-block;
                font-size: 12px;
            }}
            .detail-label {{ font-size: 11px; color: #999; text-transform: uppercase; margin-top: 15px; }}
            .detail-text {{ font-size: 14px; color: #333; line-height: 1.5; }}
            .action-btn {{ 
                display: inline-block; 
                margin-top: 20px; 
                padding: 10px 20px; 
                background: {COLORS['secondary']}; 
                color: white; 
                text-decoration: none; 
                border-radius: 4px; 
                font-size: 14px;
            }}
            
            
            #inst_selector_container {{
                position: absolute;
                top: 20px;
                left: 20px;
                z-index: 100;
                background: white;
                padding: 10px;
                border-radius: 4px;
                box-shadow: 0 2px 5px rgba(0,0,0,0.2);
            }}
            #inst_selector {{
                padding: 8px;
                font-family: 'Montserrat', sans-serif;
                font-size: 14px;
                width: 250px;
                border: 1px solid #ddd;
                border-radius: 4px;
            }}
            /* Logo Overlay */
            #logo_overlay {{ position: absolute; top: 20px; right: 20px; width: 150px; z-index: 5; pointer-events: none; }}
        </style>
    </head>
    <body>
        <div id="container">
            <div id="map_container">
                {plot_div}
                <div id="inst_selector_container">
                    <select id="inst_selector">
                        <option value="">Select an Institution...</option>
                    </select>
                </div>
                <img id="logo_overlay" src="{logo_base64}" />
            </div>
            <div id="side_panel">
                <div class="panel-header">
                    <span class="close-btn" onclick="closePanel()">&times;</span>
                    <div id="panel_title_area">
                        <h2 id="panel_title">Select Institutions</h2>
                        <span class="clear-btn" onclick="closePanel()">Clear Selection</span>
                    </div>
                </div>
                <div id="list_view" class="panel-content"></div>
                <div id="detail_view" class="panel-content detail-view"></div>
            </div>
        </div>

        <script>
            // Data Injection
            const instData = {json.dumps(inst_data)};

            
            // Populate Dropdown
            var selector = document.getElementById('inst_selector');
            Object.keys(instData).sort().forEach(function(inst) {{
                var opt = document.createElement('option');
                opt.value = inst;
                opt.innerHTML = inst;
                selector.appendChild(opt);
            }});

            // Handle Dropdown Change
            selector.addEventListener('change', function() {{
                var val = this.value;
                if (val) {{
                    selectedInsts.clear();
                    selectedInsts.add(val);
                    updatePanel();
                }} else {{
                    selectedInsts.clear();
                    closePanel();
                }}
            }});

            // Plotly Events
            var mapDiv = document.getElementById('map_div');
            var selectedInsts = new Set();

            // Handle Selection (Click, Box, Lasso)
            mapDiv.on('plotly_selected', function(eventData) {{
                if (eventData) {{
                    selectedInsts.clear();
                    eventData.points.forEach(function(pt) {{
                        var inst = pt.customdata || pt.text;
                        selectedInsts.add(inst);
                    }});
                    updatePanel();
                }} else {{
                    // Deselect event
                    selectedInsts.clear();
                    closePanel();
                }}
            }});
            
            // Handle Click (Explicitly for single clicks)
            mapDiv.on('plotly_click', function(data) {{
                if (data.points.length > 0) {{
                    var pt = data.points[0];
                    var inst = pt.customdata || pt.text;
                    selectedInsts.add(inst);
                    updatePanel();
                }}
            }});

                        // Background Click to Close
            mapDiv.on('plotly_deselect', function() {{
                closePanel();
            }});

            function updatePanel() {{
                if (selectedInsts.size === 0) {{
                    closePanel();
                    return;
                }}

                document.getElementById('side_panel').classList.add('open');
                document.getElementById('list_view').style.display = 'block';
                document.getElementById('detail_view').style.display = 'none';
                
                // Update Header
                var title = selectedInsts.size + " Institution" + (selectedInsts.size > 1 ? "s" : "") + " Selected";
                document.getElementById('panel_title_area').innerHTML = `
                    <h2 id="panel_title">${{title}}</h2>
                    <span class="clear-btn" onclick="closePanel()">Clear Selection</span>
                `;

                var listHtml = '';
                
                selectedInsts.forEach(instName => {{
                    if(instData[instName]) {{
                        listHtml += `<div class="inst-group">
                                        <div class="inst-header">${{instName}}</div>`;
                        
                        instData[instName].forEach((p, index) => {{
                            listHtml += `
                                <div class="researcher-item" onclick="showDetail('${{instName}}', ${{index}})">
                                    <div class="researcher-name">${{p.pi}}</div>
                                    <div class="project-title">${{p.title}}</div>
                                </div>
                            `;
                        }});
                        listHtml += `</div>`;
                    }}
                }});
                
                document.getElementById('list_view').innerHTML = listHtml;
            }}

            function closePanel() {{
                document.getElementById('side_panel').classList.remove('open');
                selectedInsts.clear();
                // Clear Plotly Selection Visuals
                Plotly.restyle(mapDiv, {{selectedpoints: [null]}});
            }}

            function showDetail(instName, index) {{
                var p = instData[instName][index];
                
                document.getElementById('list_view').style.display = 'none';
                document.getElementById('detail_view').style.display = 'block';
                
                // Update Header with Back Button
                document.getElementById('panel_title_area').innerHTML = `
                    <div class="back-btn" onclick="updatePanel()">&#8592; Back to List</div>
                    <h2>${{p.pi}}</h2>
                `;
                
                var detailHtml = `
                    <div class="detail-label">Institution</div>
                    <div class="detail-text">${{instName}}</div>

                    <div class="detail-label">Project Title</div>
                    <div class="detail-text"><b>${{p.title}}</b></div>
                    
                    <div class="detail-label">Contact</div>
                    <div class="detail-text">${{p.email}}</div>
                    
                    <div class="detail-label">Year</div>
                    <div class="detail-text">${{p.year}}</div>
                    
                    <div class="detail-label">Key Findings / Summary</div>
                    <div class="detail-text">${{p.summary}}</div>
                    
                    <a href="${{p.link}}" target="_blank" class="action-btn">View Research</a>
                `;
                
                document.getElementById('detail_view').innerHTML = detailHtml;
            }}
        </script>
    </body>
    </html>
    """

    with open(f'{output_dir}/interactive_institutions_map.html', 'w') as f:
        f.write(html_content)
    
    print("✓ Saved interactive_institutions_map.html (Multi-Select Version)")

create_custom_interactive_map()

✓ Saved interactive_institutions_map.html (Multi-Select Version)


In [5]:
# --- 3. INTERACTIVE KEYWORD CHART ---

def create_keyword_chart():
    # Prepare Data
    def get_keywords(df_sub):
        d = df_sub[df_sub['project_year'].between(2015, 2024)]
        return d['science_priority'].value_counts().reset_index()

    t_kw = get_keywords(df_work)
    t_kw.columns = ['Topic', 'Count']
    
    s_kw = get_keywords(df_seed_only)
    s_kw.columns = ['Topic', 'Count']

    # Create Sunburst-like Pie Charts
    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{'type': 'domain'}, {'type': 'domain'}]],
        subplot_titles=("Total Projects", "Seed Funding")
    )

    fig.add_trace(go.Pie(
        labels=t_kw['Topic'], values=t_kw['Count'],
        name='Total',
        marker_colors=px.colors.qualitative.Prism,
        hole=0.4
    ), 1, 1)

    fig.add_trace(go.Pie(
        labels=s_kw['Topic'], values=s_kw['Count'],
        name='Seed',
        marker_colors=px.colors.qualitative.Prism,
        hole=0.4
    ), 1, 2)

    fig.update_layout(
        title=dict(
            text="<b>Research Topic Distribution (2015-2024)</b>",
            font=dict(size=24, family=FONT_FAMILY, color=COLORS['primary']),
            y=0.95
        ),
        font=dict(family=FONT_FAMILY),
        height=600
    )

    if logo_base64:
        fig.add_layout_image(
            dict(
                source=logo_base64,
                xref="paper", yref="paper",
                x=1, y=1.15,
                sizex=0.15, sizey=0.15,
                xanchor="right", yanchor="top"
            )
        )

    fig.write_html(f'{output_dir}/interactive_topic_distribution.html')
    print("✓ Saved interactive_topic_distribution.html")

create_keyword_chart()

✓ Saved interactive_topic_distribution.html
